In [4]:
pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1321 sha256=99c09c80ae5769c863f22dda72d270292305fdaadbd73cc45b2b6d4e4b3ee5b6
  Stored in directory: C:\Users\RIEKS\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [3]:
import random
from sklearn import preprocessing  # pip install sklearn ... if you don't have it!


ModuleNotFoundError: No module named 'sklearn'

In [1]:
import import_ipynb
import preposes

importing Jupyter notebook from preposes.ipynb
importing Jupyter notebook from indicators.ipynb
importing Jupyter notebook from add_indicators.ipynb
importing Jupyter notebook from dataset.ipynb
 0    14899
 1     5113
-1     4809
Name: pred, dtype: int64
                       Close      Low      min      max   future  pred
date                                                                  
2015-01-08 02:00:00  1.18258  1.18255  1.18037  1.18312  1.18202     0
2015-01-08 03:00:00  1.18218  1.18188  1.17918  1.18312  1.17945    -1
2015-01-08 04:00:00  1.18270  1.18174  1.17801  1.18291  1.17812    -1
2015-01-08 05:00:00  1.18225  1.18205  1.17625  1.18267  1.17689    -1
2015-01-08 06:00:00  1.18213  1.18170  1.17625  1.18240  1.17735    -1
2015-01-08 07:00:00  1.18205  1.18178  1.17543  1.18228  1.17760    -1
2015-01-08 08:00:00  1.18202  1.18037  1.17543  1.18228  1.17875    -1
2015-01-08 09:00:00  1.17945  1.17918  1.17543  1.18078  1.17978     0
2015-01-08 10:00:00  1.17812  1.17

In [2]:
df = preposes.df

,Open,High,Low,Close,Volume,MA_10,MA_21,MA_50,EMA_8,EMA_25,EMA_100,ROC_9,ROC_14,ROC_25,BollingerB_9,Bollinger%b_9,BollingerB_12,Bollinger%b_12,pred
date,,,,,,,,,,,,,,,,,,,
2015-01-08 02:00:00,1.18387,1.18405,1.18255,1.18258,3998.5400,1.183380,1.184050,1.188239,1.183338,1.184824,1.189837,-0.000296,-0.000879,-0.004319,0.002331,0.156510,0.004081,0.413861,0
2015-01-08 03:00:00,1.18258,1.18272,1.18188,1.18218,3099.3799,1.183393,1.183781,1.187997,1.183081,1.184621,1.189662,-0.001098,-0.000693,-0.004061,0.002726,0.108041,0.004014,0.323454,-1
2015-01-08 04:00:00,1.18219,1.18312,1.18174,1.18270,3346.9900,1.183370,1.183592,1.187769,1.182996,1.184473,1.189503,-0.001958,0.000668,-0.004134,0.002850,0.304991,0.002769,0.340271,-1
2015-01-08 05:00:00,1.18270,1.18291,1.18205,1.18225,2227.0701,1.183247,1.183284,1.187522,1.182830,1.184302,1.189339,-0.000896,0.001703,-0.004471,0.002172,0.188619,0.002687,0.188570,-1
2015-01-08 06:00:00,1.18225,1.18267,1.18170,1.18213,3091.7000,1.182958,1.183060,1.187276,1.182675,1.184135,1.189176,-0.001495,0.000068,-0.004807,0.002368,0.218415,0.002889,0.194719,-1


In [ ]:
def preprocess_df(df):

    for col in df.columns:  # go through all of the columns
        if col != "pred":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic. Those nasty NaNs love to creep in.

    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.
    
    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets
    hold = []
    
    for seq, target in sequential_data:  # iterate over the sequential data
        if target == -1:
            sells.append([seq, target])
        elif target == 0:  # if it's a "not buy"
            hold.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!
    random.shuffle(hold)

    lower = min(min(len(buys), len(sells)),len(hold))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.
    hold = hold[:lower]
    
    sequential_data = buys+sells+hold  # add them together
    random.shuffle(sequential_data)
    
    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y